In [4]:
from torch import optim
from torchvision.models import efficientnet_b1, EfficientNet_B1_Weights
import torch
import random
import numpy as np
import torch.nn as nn
import albumentations as Albu
import pandas as pd
from torch.utils.data.sampler import RandomSampler
from warmup_scheduler import GradualWarmupScheduler
import os
from utils.dataset import PandasDataset
from utils.metrics import model_checkpoint
from utils.train import train_model
from utils.models import EfficientNetApi

In [5]:
seed = 42
shuffle = True
batch_size = 6
num_workers = 4
output_classes = 5
init_lr = 3e-4
warmup_factor = 2
warmup_epochs = 1
n_epochs = 50
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
loss_function = nn.BCEWithLogitsLoss()

torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)

ROOT_DIR = '../..'

data_dir = '../../../dataset'
images_dir = os.path.join(data_dir, 'tiles')

Using device: cuda


In [6]:
load_model = efficientnet_b1(
     weights=EfficientNet_B1_Weights.DEFAULT
)
model = EfficientNetApi(model=load_model, output_dimensions=output_classes, dropout_rate=0.6)
model = model.to(device)

In [7]:
print("Using device:", device)
loss_function = nn.BCEWithLogitsLoss()

torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)

Using device: cuda


In [8]:
df_train_ = pd.read_csv(f"{ROOT_DIR}/data/train_5fold.csv")
df_train_.columns = df_train_.columns.str.strip()
train_indexes = np.where((df_train_['fold'] != 3))[0]
valid_indexes = np.where((df_train_['fold'] == 3))[0]
#
df_train = df_train_.loc[train_indexes]
df_val = df_train_.loc[valid_indexes]
df_test = pd.read_csv(f"{ROOT_DIR}/data/test.csv")

#### view data

In [9]:
(df_train.shape, df_val.shape, df_test.shape)

((7219, 5), (1805, 5), (1592, 4))

In [10]:
transforms = Albu.Compose([
    Albu.Transpose(p=0.5),
    Albu.VerticalFlip(p=0.5),
    Albu.HorizontalFlip(p=0.5),
])

In [11]:
df_train.columns = df_train.columns.str.strip()

train_dataset = PandasDataset(images_dir, df_train, transforms=transforms)
valid_dataset = PandasDataset(images_dir, df_val, transforms=None)
test_dataset = PandasDataset(images_dir, df_test, transforms=None)

In [12]:
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, num_workers=num_workers, sampler=RandomSampler(train_dataset)
)
valid_loader = torch.utils.data.DataLoader(
    valid_dataset, batch_size=batch_size, num_workers=num_workers, sampler = RandomSampler(valid_dataset)
)
test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=batch_size, num_workers=num_workers, sampler = RandomSampler(test_dataset)
)

In [13]:
optimizer = optim.Adam(model.parameters(), lr = init_lr / warmup_factor)
scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, n_epochs - warmup_epochs)
scheduler = GradualWarmupScheduler(optimizer, multiplier = warmup_factor, total_epoch = warmup_epochs, after_scheduler=scheduler_cosine)

In [14]:
train_model(
    model=model,
    epochs=n_epochs,
    optimizer=optimizer,
    scheduler=scheduler,
    train_dataloader=train_loader,
    valid_dataloader=valid_loader,
    checkpoint=model_checkpoint,
    device=device,
    loss_function=loss_function,
    path_to_save_metrics="logs/b1.txt",
    path_to_save_model="models/b1.pth",
    patience=5,
)

Epoch 1/50



100%|██████████| 301/301 [01:55<00:00,  2.61it/s]


VAL_LOSS     0.292
VAL_ACC      Mean: 50.277 | Std: 1.151 | 95% CI: [48.421, 52.078]
VAL_KAPPA    Mean: 0.768 | Std: 0.012 | 95% CI: [0.749, 0.786]
VAL_F1       Mean: 0.434 | Std: 0.012 | 95% CI: [0.415, 0.454]
VAL_RECALL   Mean: 0.434 | Std: 0.011 | 95% CI: [0.416, 0.452]
VAL_PRECISION Mean: 0.528 | Std: 0.013 | 95% CI: [0.506, 0.550]
Salvando o melhor modelo... 0.0 -> 0.7679253809124893
Epoch 2/50



100%|██████████| 301/301 [01:55<00:00,  2.60it/s]
/home/woshington/Projects/Doutorado/repo/.venv/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:1087: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  _warn_get_lr_called_within_step(self)


VAL_LOSS     0.290
VAL_ACC      Mean: 54.809 | Std: 1.150 | 95% CI: [52.909, 56.676]
VAL_KAPPA    Mean: 0.768 | Std: 0.012 | 95% CI: [0.748, 0.787]
VAL_F1       Mean: 0.456 | Std: 0.012 | 95% CI: [0.436, 0.475]
VAL_RECALL   Mean: 0.461 | Std: 0.011 | 95% CI: [0.443, 0.478]
VAL_PRECISION Mean: 0.558 | Std: 0.013 | 95% CI: [0.535, 0.579]
Salvando o melhor modelo... 0.7679253809124893 -> 0.7681001925213348
Epoch 3/50



...

VAL_LOSS     0.475
VAL_ACC      Mean: 63.332 | Std: 1.139 | 95% CI: [61.496, 65.266]
VAL_KAPPA    Mean: 0.814 | Std: 0.012 | 95% CI: [0.794, 0.834]
VAL_F1       Mean: 0.568 | Std: 0.012 | 95% CI: [0.549, 0.588]
VAL_RECALL   Mean: 0.559 | Std: 0.012 | 95% CI: [0.540, 0.579]
VAL_PRECISION Mean: 0.592 | Std: 0.013 | 95% CI: [0.572, 0.613]
Epoch 13/50



100%|██████████| 301/301 [02:00<00:00,  2.51it/s]


VAL_LOSS     0.539
VAL_ACC      Mean: 64.724 | Std: 1.125 | 95% CI: [62.881, 66.537]
VAL_KAPPA    Mean: 0.814 | Std: 0.013 | 95% CI: [0.792, 0.835]
VAL_F1       Mean: 0.581 | Std: 0.012 | 95% CI: [0.560, 0.601]
VAL_RECALL   Mean: 0.572 | Std: 0.012 | 95% CI: [0.552, 0.591]
VAL_PRECISION Mean: 0.606 | Std: 0.013 | 95% CI: [0.585, 0.626]
Epoch 14/50



100%|██████████| 301/301 [01:58<00:00,  2.54it/s]


VAL_LOSS     0.505
VAL_ACC      Mean: 64.774 | Std: 1.139 | 95% CI: [62.825, 66.593]
VAL_KAPPA    Mean: 0.828 | Std: 0.012 | 95% CI: [0.809, 0.848]
VAL_F1       Mean: 0.591 | Std: 0.012 | 95% CI: [0.571, 0.612]
VAL_RECALL   Mean: 0.586 | Std: 0.012 | 95% CI: [0.567, 0.607]
VAL_PRECISION Mean: 0.600 | Std: 0.013 | 95% CI: [0.580, 0.622]

Early stopping at epoch 14. No improvement for 5 epochs.
Best epoch: 9 with kappa: 0.8390


# tests

In [15]:
from utils.metrics import evaluation, format_metrics
model.load_state_dict(
    torch.load(f"models/b1.pth")
)
response = evaluation(model, test_loader, device)
result = format_metrics(response[0])
print(result)

100%|██████████| 266/266 [01:36<00:00,  2.75it/s]


VAL_ACC      Mean: 63.799 | Std: 1.159 | 95% CI: [61.935, 65.704]
VAL_KAPPA    Mean: 0.834 | Std: 0.013 | 95% CI: [0.812, 0.854]
VAL_F1       Mean: 0.572 | Std: 0.013 | 95% CI: [0.553, 0.594]
VAL_RECALL   Mean: 0.569 | Std: 0.013 | 95% CI: [0.550, 0.591]
VAL_PRECISION Mean: 0.584 | Std: 0.013 | 95% CI: [0.563, 0.606]
